In [39]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
from matplotlib import pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
import seaborn as sns
import cufflinks as cf
cf.set_config_file(offline=False, world_readable=True, theme='ggplot')
from DataFrame import *
from netCDF4 import Dataset as nc, num2date

In [48]:
sns.set_style("whitegrid")
cpolFile = os.path.join(os.environ['HOME'], 'Data', 'Extremes', 'CPOL', 'CPOL_area_avg_precip.pkl')
largeScaleFile = os.path.join(os.environ['HOME'], 'Data', 'Extremes', 'CPOL', 'CPOL-large-scale_forcing.nc')
popeFile = os.path.join(os.environ['HOME'], 'Data', 'Extremes', 'CPOL', 'Pope_regimes.pkl')

#### Load the datasets for CPOL rainfall and Large-scale forcing

In [49]:
cpolFrame = pd.read_pickle(cpolFile)
with nc(largeScaleFile) as h5:
    precip = h5.variables['prec_srf'][:]
    plev = h5.variables['lev'][:]
    omega_600 = h5.variables['omega'][:,np.argmin(np.fabs(600-plev))]
    times = pd.DatetimeIndex(num2date(h5.variables['time'][:],h5.variables['time'].units))
    largeScale = pd.DataFrame(omega_600,
        index=times,columns=['Omega'])

#### Make 10 Min Cpol data 6 hourly

In [206]:
cpol10m = cpolFrame['all']
cpol1H = convScale(cpolFrame['all']/6,'sum','1H')
cpol6H = dict(CpolRain = convScale(cpol1H,'mean','6H'))
cpol1H = dict(CpolRain =cpol1H)
cpol6H['Regime'] = convScale(cpolFrame['regime'],'mean','6H')
cpol1H['Regime'] = convScale(cpolFrame['regime'],'mean','1H')
cpol6H = pd.DataFrame(cpol6H)
cpol1H = pd.DataFrame(cpol1H)


### The Timeseries of the 6 hly Cpol Data (Pope Regimes added)

In [51]:
regime = convScale(cpol6H['Regime'],'mean','1D')

In [52]:
periods = partition(pd.read_pickle(popeFile))

In [53]:
color = {1. : 'orange', 2. : 'green', 3. : 'red', 4. : 'purple', 5. : 'brown'}

In [54]:
shapes = get_shapes(color,periods)

In [55]:
data = [ go.Scatter(x=cpol6H['CpolRain'].index,y=cpol6H['CpolRain'].values) ]

In [56]:
annotations = []
yy = 1
for reg, co in color.items():
    annotations.append(
        dict(
            x=1.08,
            y=yy,
            xref='paper',
            yref='paper',
            text='Regime %1.0f'%reg,
            showarrow=False,
            font=dict(
                family='sans',
                size=12,
                color='white'
            ),
            align='center',
            bordercolor='#c7c7c7',
            borderwidth=1,
            borderpad=4,
            bgcolor=co,
            opacity=0.8
        )
    )
    yy -= 0.15

In [57]:
layout = dict(shapes=shapes,yaxis=dict(title='6h avg Rainfall [mm/h]'),annotations=annotations)

In [58]:
py.iplot({'data': data, 'layout': layout}, filename='rain_timeseries')

### The Number of Extreme Events by Quantile

In [59]:
quantiles = [80,85,90,95,99]
extremeCount = []
for quan in quantiles:
    extremeCount.append(cpol6H['CpolRain'][cpol6H['CpolRain']>cpol6H['CpolRain'].quantile(quan/100.)].count())

In [60]:
extremeCount = pd.Series(extremeCount,index=quantiles)

In [61]:
extremeCount.iplot(kind='bar', filename='extreme_Count',yTitle='Number of Events Above Quantile',xTitle='Quantile',
                  title='Number of Events vs. Quantile (6 hly)')

### Extreme Rainfall Events (>90)  per Pope Regime

In [62]:
cpol6H_2 = pd.concat([cpol6H.loc[largeScale.index],largeScale],axis=1).dropna()
extreme = cpol6H_2.loc[cpol6H['CpolRain']>cpol6H['CpolRain'].quantile(90/100.)]
extreme_2 = cpol6H.loc[cpol1H['CpolRain']>cpol1H['CpolRain'].quantile(90/100.)]

In [63]:
regCount = []
for reg in range(1,6):
    regCount.append(extreme_2['CpolRain'].loc[extreme_2['Regime']==reg].count())
regCount = pd.Series(regCount,index=range(1,6))
regCount.iplot(kind='bar', filename='reg_Count',yTitle='Number of Events in Rainfall Regime',xTitle='Regime',
                  title='Number of Events vs. Rainfall Regime')

In [66]:
compare_1 = [] 
compare_1.append({
            "type": 'violin',
            "y": extreme_2['CpolRain'].values,
            "name": 'all Regimes',
            "box": {
                "visible": True
            },
            "meanline": {
                "visible": True
            }
        })
for reg in range(1,6):
        compare_1.append({
            "type": 'violin',
            "y": extreme_2['CpolRain'].loc[extreme_2['Regime'] == reg].values,
            "name": "Regime %i" %reg,
            "box": {
                "visible": True
            },
            "meanline": {
                "visible": True
            }
        })
        
    
fig_1 = {
    "data": compare_1,
    "layout" : {
        "title": "Radar rain in Different Rainfall Regimes",
        "yaxis": {
            "zeroline": False,
            "title": "6 Hour Rain-Rate [mm/h]"
            
        }
    }
} 

In [65]:
py.iplot(fig_1, filename='rainfall_violine',validate=False)

### Large-scale variables in Extreme Rainfall Situations
##### The Distribution of $\omega_{600}$ for Rainfall Events Above the 90th Percentile 

In [67]:
largeScales = [e for e in extreme.columns if e.lower() not in ('cpolrain','regime')]
compare = {}
tt = 1
for var in largeScales:
    compare[var]=[]
    compare[var].append({
            "type": 'violin',
            "y": extreme[var].values,
            "name": 'all Regimes',
            "box": {
                "visible": True
            },
            "meanline": {
                "visible": True
            }
        })
    for reg in range(1,6):
        compare[var].append({
            "type": 'violin',
            "y": extreme[var].loc[extreme['Regime'] == reg].values,
            "name": "Regime %i" %reg,
            "box": {
                "visible": True
            },
            "meanline": {
                "visible": True
            }
        })
        
    
fig = {
    "data": compare['Omega'],
    "layout" : {
        "title": "$ \\omega_{600} \\text{ in Different Rainfall Regimes}$",
        "yaxis": {
            "zeroline": False,
            "title": "$ \\omega_{600{}}  \\text{ [hPa/h]}$"
            
        }
    }
} 

In [68]:
py.iplot(fig, filename='omega_violine',validate=False)

### Powerlaw of Rainfall aka Self-Organized Crticality
First test if rainfall follows as powerlaw. For this bin the rainfall-rate (0.01 steps) and rank them

In [441]:
cpol1hBins = cpol10m.round(2).dropna()
cpol1hBins = cpol1hBins.loc[cpol1hBins>0]
#cpol1hBins = cpol1hBins.loc[cpol1hBins<10]

In [432]:
unique = np.unique(cpol1hBins.values)

In [433]:
rank = [cpol1hBins.loc[cpol1hBins == i].count() for i in unique]

In [442]:
m, t= -87.54385964912281, 701.3508771929825
x=np.linspace(1,8,100)
y=m*x+t

In [440]:
data = [ go.Scatter(x=unique,y=rank),go.Scatter(x=[x[0],x[-1]],y=[56*y[0],y[-1]])]
layout = dict(yaxis=dict(title='log(Number of Occurrances)',type='log'),
              xaxis=dict(title='log(Rain-rate)',type='log'))
py.iplot({'data': data, 'layout': layout}, filename='rain_rank')